![](./图片2.png)

**CPU**：中央处理器，主要包括控制器和运算器。

**GPU**：图形处理器，处理统一的，无依赖的大规模数据运算。

CPU 的计算单元少，GPU 的计算单元多。

<br/>

# 1. to() 方法

**使用 `.to()` 方法将数据和模型从 CPU 转移到 GPU**：
```
device = torch.device("cuda")
tensor = tensor.to(device)
module.to(device)
```

**注意：`tensor.to()` 执行的不是 inplace 操作，存储地址改变了，因此需要赋值；`module.to()` 执行的是 inplace 操作，在原存储地址操作，存储地址不变。**

**使用 `.to()` 方法转换数据类型**：
```
x = torch.ones((3,3))
x = x.to(torch.float64)
```

<br/>

# 2. `torch.cuda` 的常用方法
- torch.cuda.device_count()：返回当前可见可用的 GPU 数量
- torch.cuda.get_device_name()：获取 GPU 名称
- torch.cuda.manual_seed()：为当前 GPU 设置随机种子
- torch.cuda.manual_seed_all()：为所有可见可用 GPU 设置随机种子
- torch.cuda.set_device()：设置主 GPU 为哪一个物理 GPU，此方法不推荐使用
- **推荐** `os.environ.setdefault("CUDA_VISIBLE_DEVICES", "1, 3, 2")`：设置 python 脚本可见 GPU，第一个可见 GPU 为逻辑 GPU 0，通常默认逻辑 GPU 0 为主 GPU。

在 PyTorch 中，有物理 GPU 和逻辑 GPU 之分，物理 GPU 就是实打实的所有 GPU，设置的 python 脚本可见的 GPU 就是逻辑 GPU。

如果我们有 4 个 GPU，执行 `os.environ.setdefault("CUDA_VISIBLE_DEVICES", "1, 3, 2")`，那么可见 GPU 数量只有 3 个（第二个、第四个和第三个），对应关系为：逻辑 GPU 0 为物理 GPU 2，逻辑 GPU 1 为物理 GPU 4，逻辑 GPU 2 为物理 GPU 3。

**为什么需要设置可见 GPU**：因为可能有很多用户和任务在同时使用 GPU，设置可见 GPU，可以合理分配 GPU。

<br/>

# 3. 多 GPU 的分发并行机制

主 GPU 的概念与多 GPU 的分发并行机制有关：**主 GPU 分发计算任务 ➡ 并行计算 ➡ 主 GPU 回收计算结果**。

`torch.nn.DataParallel(module, device_ids=None, output_device=None, dim=0)`

**功能**：包装模型，实现分发并行机制。

**主要参数**：
- module：需要包装分发的模型
- device_ids：可分发的 GPU，默认分发到所有可见可用的 GPU，每个 GPU 计算的数据量为 $\frac{batchsize}{GPU 数量}$，实现并行计算。
- output_device：结果输出设备

**设置可见 GPU 的方法有两种**：手动选择和依内存情况自动设置，无论手动还是自动，第一个可见 GPU 为主 GPU。

**代码**：

    import os
    import numpy as np
    import torch
    import torch.nn as nn

    # ============================ 方法一：手动选择 GPU =======================================
    gpu_list = [2, 3]
    gpu_list_str = ','.join(map(str, gpu_list))
    os.environ.setdefault("CUDA_VISIBLE_DEVICES", gpu_list_str)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    # ============================ 方法二：依内存情况自动设置 GPU ==============================
    # 查询当前 GPU 剩余内存
    def get_gpu_memory():
        import platform
        if 'Windows' != platform.system():
            import os
            os.system('nvidia-smi -q -d Memory | grep -A4 GPU | grep Free > tmp.txt')
            memory_gpu = [int(x.split()[2]) for x in open('tmp.txt', 'r').readlines()]
            os.system('rm tmp.txt')
        else:
            memory_gpu = False
            print("显存计算功能暂不支持windows操作系统")
        return memory_gpu


    gpu_memory = get_gpu_memory()

    # 获取 GPU 剩余内存从大到小排序的物理 GPU 索引，GPU 剩余内存最大的 GPU 作为主 GPU
    print("\ngpu free memory: {}".format(gpu_memory))
    gpu_list = np.argsort(gpu_memory)[::-1]
    gpu_list_str = ','.join(map(str, gpu_list))
    os.environ.setdefault("CUDA_VISIBLE_DEVICES", gpu_list_str)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    # =================================== 训练 ==============================================
    batch_size = 16
    num_epoches = 10
    
    # data
    ···

    # model
    net = ···
    net = nn.DataParallel(net)  # 包装模型
    net.to(device)  # 将模型转移至 GPU
    
    # training
    for epoch in range(num_epoches):
        for i, data in enumerate(train_loader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)  # 将数据转移至 GPU
            outputs = net(inputs)
            print("model outputs.size: {}".format(outputs.size()))
        
    print("CUDA_VISIBLE_DEVICES :{}".format(os.environ["CUDA_VISIBLE_DEVICES"]))
    print("device_count :{}".format(torch.cuda.device_count()))

<br/>

# 4. 提高 GPU 的利用率

`nvidia-smi` 命令查看可以 GPU 的利用率，Memory Usage 表示显存的使用率，Volatile GPU-Util 表示 GPU 实际运算能力的利用率。

虽然使用 GPU 可以加速训练模型，但是如果 GPU 的 Memory Usage 和 Volatile GPU-Util 太低，表示并没有充分利用 GPU。因此，使用 GPU 训练模型，需要尽量提高 GPU 的 Memory Usage 和 Volatile GPU-Util 这两个指标，可以更进一步加速训练过程。

如何提高这两个指标：

**Memory Usage**：这个指标主要是由模型大小，以及数据量的大小决定的。我们主要调整的是每个 batch 训练的数据量的大小，也就是 batch_size。在模型结构固定的情况下，尽量将batch size 设置得比较大，充分利用 GPU 的内存。

**Volatile GPU-Util**：上面设置比较大的 batch size 可以提高 GPU 的内存使用率，却不一定能提高 GPU 运算能力的利用率。根据前面代码可知，我们的数据首先是读取到 CPU 中，然后在循环训练的时候通过 `tensor.to()` 将数据从 CPU 加载到 GPU 中，如果 batch size 得比较大，那么在 Dataset 和 DataLoader，CPU 处理一个 batch 的数据就会很慢，这时就会发现 Volatile GPU-Util 的值会在 0%，20%，70%，95%，0% 之间不断变化（nvidia-smi命令查看可以 GPU 的利用率，但不能动态刷新显示。如果想每隔一秒刷新显示 GPU 信息，可以使用 `watch -n 1 nvidia-smi`）。其实这是因为 GPU 处理数据非常快，而 CPU 处理数据较慢。GPU 每接收到一个 batch 的数据，使用率就逐渐升高，处理完这个 batch 的数据后，使用率又逐渐降低，等到 CPU 把下一个 batch 的数据传过来。

**可以通过设置 Dataloader 的两个参数解决上述问题**：
- num_workers：默认只使用一个 CPU 读取和处理数据，可以设置为 4、8、16 等参数。但线程数并不是越大越好，因为多核处理需要把数据分发到每个 CPU，处理完成后需要从多个 CPU 收集数据，这个过程也是需要时间的。如果设置 num_workers 过大，分发和收集数据等操作占用了太多时间，反而会降低效率。
- pin_memory：如果显存较大，建议设置为 True。设置为 True，表示把数据直接映射到 GPU 的相关内存块上，省掉了一点数据传输时间。设置为 False，表示从 CPU 传入到缓存 RAM 里面，再给传输到 GPU 上。

<br/>

# 5. GPU 相关的报错

1. 如果模型是在 GPU 上训练后保存的，然后在无 GPU 的设备上直接加载模型，即 `state_dict_load = torch.load(path_state_dict)`，就会报错。

   报错信息：`RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.`

    **解决方法是设置 `map_location="cpu"`**：`state_dict_load = torch.load(path_state_dict, map_location="cpu")`

<br/>

2. 如果模型是在 GPU 上训练后保存的，模型会经过 `net = nn.DataParallel(net)` 包装，包装后模型所有网络层的名称前面都会加上 module，例如 module.linears。在 GPU 上训练后保存的模型再次加载时，如果不计划在 GPU 上使用，提前创建的模型对象不使用 `nn.DataParallel()` 包装，就会加载失败报错，因为两者的 state_dict 中模型网络层的名称对应不上。

    报错信息：`Missing key(s) in state_dict: xxxxxxxxxx，Unexpected key(s) in state_dict:xxxxxxxxxx`
   
    **解决方法**：遍历 state_dict_load 的 key（模型网络层的名称），如果 key 的名字是以 module. 开头，则去掉 module.。
   
    **代码如下**：
   
        # 提前创建一个模型对象
        net = FooNet(neural_num=3, layers=3)
    
        path_state_dict = "./model_in_multi_gpu.pkl"  # 模型是在 GPU 上训练后保存的
        state_dict_load = torch.load(path_state_dict, map_location="cpu")  # 在 CPU 上加载模型，设置 map_location="cpu"
    
        # 去掉 state_dict_load 中模型网络层名称开头的 module.
        from collections import OrderedDict
        new_state_dict = OrderedDict()
        for k, v in state_dict_load.items():
            namekey = k[7:] if k.startswith('module.') else k
            new_state_dict[namekey] = v
   
        # 去掉模型网络层名称的 module. 之后的新的 new_state_dict 中模型网络层的名称能对应上 net 的 state_dict 中模型网络层的名称，此时就可以将参数加载到 net 中
        net.load_state_dict(new_state_dict)